In [6]:
import pandas as pd
import numpy as np

# 1. 读取并合并三个CSV文件
df_2015 = pd.read_csv('2015年国内主要城市年度数据.csv', encoding='utf-8-sig')
df_2016 = pd.read_csv('2016年国内主要城市年度数据.csv', encoding='utf-8-sig')
df_2017 = pd.read_csv('2017年国内主要城市年度数据.csv', encoding='utf-8-sig')

In [7]:
# 纵向连接三个数据框
merged_df = pd.concat([df_2015, df_2016, df_2017], ignore_index=True)

In [8]:
# 2. 按年份分组聚合
yearly_data = merged_df.groupby('年份')

In [9]:
# 3. 计算每年的国内生产总值总和
gdp_by_year = yearly_data['国内生产总值'].sum()
print("各年份国内生产总值总和:")
print(gdp_by_year.to_string())
print("\n")

各年份国内生产总值总和:
年份
2015    277958.37
2016    300696.10
2017    332562.43




In [10]:
# 4. 处理缺失值（填充为0）
filled_df = merged_df.fillna(0)
# 检查填充结果
print("缺失值处理后的数据示例:")
print(filled_df.head().to_string())

缺失值处理后的数据示例:
     地区    年份    国内生产总值  第一产业增加值  第二产业增加值   第三产业增加值  社会商品零售总额    货物进出口总额    年末总人口  在岗职工平均工资  普通高等学校在校学生数  医院、卫生院数  房地产开发投资额
0    北京  2015  23014.59   140.21  4542.64  18331.74   10338.0  319416.16  1345.20    113073        60.36      701   4177.05
1    天津  2015  16538.19   208.82  7704.22   8625.15    5257.3  114282.80  1026.90     81486        51.29      661   1871.55
2   石家庄  2015   5440.60   494.44  2452.40   2493.77    2693.0   12160.29  1028.84     54441        41.98      393    965.13
3    太原  2015   2735.34    37.40  1020.18   1677.77    1540.8   10677.38   367.39     60516        42.14      247    597.83
4  呼和浩特  2015   3090.52   126.23   867.08   2097.21    1353.5    2072.75   238.58     53698        23.52      174    509.05


In [21]:
# 5. 修正后的增长率计算方式
def calculate_growth(df):
    # 先筛选出每个城市三年的数据
    city_gdp = df.pivot(index='年份', columns='地区', values='国内生产总值')

    # 计算年均复合增长率
    growth_rates = {}
    for city in city_gdp.columns:
        gdp_values = city_gdp[city]
        if len(gdp_values) == 3 and gdp_values[2015] > 0:
            cagr = ((gdp_values[2017] / gdp_values[2015]) ** (1/2) -(1))
            growth_rates[city] = cagr * 100

    return pd.DataFrame({
        '城市': growth_rates.keys(),
        'GDP年均增长率(%)': growth_rates.values()
    }).sort_values('GDP年均增长率(%)', ascending=False)

growth_rates = calculate_growth(merged_df)

print("GDP年均增长率最高的5个城市:")
print(growth_rates.head(5).to_string(index=False))
print("\nGDP年均增长率最低的5个城市:")
print(growth_rates.tail(5).to_string(index=False))
print("\n")

GDP年均增长率最高的5个城市:
城市  GDP年均增长率(%)
西安    13.474172
成都    13.398259
深圳    13.355030
合肥    12.889341
拉萨    12.788795

GDP年均增长率最低的5个城市:
  城市  GDP年均增长率(%)
 哈尔滨     5.118674
乌鲁木齐     2.109128
  大连    -2.406989
呼和浩特    -5.777610
  沈阳   -10.195788




In [25]:
def normalize_hospitals(df):
    result = df.copy()

    # 按年份分组计算每组的min-max
    grouped = df.groupby('年份')['医院、卫生院数']
    min_values = grouped.min().to_dict()
    max_values = grouped.max().to_dict()

    # 应用归一化
    for year in df['年份'].unique():
        year_mask = df['年份'] == year
        min_val = min_values[year]
        max_val = max_values[year]
        diff = max_val - min_val

        # 处理除零情况
        if diff > 0:
            result.loc[year_mask, '医院标准化值'] = (df.loc[year_mask, '医院、卫生院数'] - min_val) / diff
        else:
            result.loc[year_mask, '医院标准化值'] = 0.5  # 当所有值相同时设为中间值

    return result

# 应用归一化
normalized_df = normalize_hospitals(merged_df)

# 按年份和地区展示结果
print("各城市医疗资源标准化结果（按年份比较）:")
for year in sorted(normalized_df['年份'].unique()):
    print(f"\n年份 {year}:")
    yearly_data = normalized_df[normalized_df['年份'] == year]
    print(yearly_data[['地区', '医院、卫生院数', '医院标准化值']]
          .sort_values('医院标准化值', ascending=False)
          .head(10)
          .to_string(index=False))

各城市医疗资源标准化结果（按年份比较）:

年份 2015:
 地区  医院、卫生院数   医院标准化值
 重庆     1568 1.000000
 成都      768 0.470199
 北京      701 0.425828
 天津      661 0.399338
 上海      644 0.388079
 合肥      470 0.272848
哈尔滨      462 0.267550
 西安      395 0.223179
石家庄      393 0.221854
 昆明      392 0.221192

年份 2016:
 地区  医院、卫生院数   医院标准化值
 重庆     1606 1.000000
 成都      866 0.521345
 北京      713 0.422380
 上海      656 0.385511
 天津      571 0.330530
哈尔滨      471 0.265847
 合肥      462 0.260026
石家庄      425 0.236093
 昆明      414 0.228978
 西安      392 0.214748

年份 2017:
 地区  医院、卫生院数   医院标准化值
 成都      888 1.000000
 重庆      749 0.838372
 北京      656 0.730233
 天津      426 0.462791
 青岛      410 0.444186
 上海      363 0.389535
 武汉      354 0.379070
 西安      329 0.350000
哈尔滨      310 0.327907
 昆明      308 0.325581


In [15]:
# 7. 提取四大城市数据
big4 = ['北京', '上海', '广州', '深圳']
big4_data = merged_df[merged_df['地区'].isin(big4)][['地区', '年份', '国内生产总值', '社会商品零售总额']]
big4_pivot = big4_data.pivot(index='地区', columns='年份', values=['国内生产总值', '社会商品零售总额'])
big4_pivot.to_csv('四大城市经济指标.csv', encoding='utf-8-sig')

print("四大城市2015-2017年数据已保存到: 四大城市经济指标.csv")
print("数据预览:")
print(big4_pivot)

四大城市2015-2017年数据已保存到: 四大城市经济指标.csv
数据预览:
      国内生产总值                     社会商品零售总额                  
年份      2015      2016      2017     2015     2016     2017
地区                                                         
上海  25123.45  28178.65  30632.99  10131.5  10946.6  11830.3
北京  23014.59  25669.13  28014.94  10338.0  11005.1  11575.4
广州  18100.41  19547.44  21503.15   7988.0   8706.5   9402.6
深圳  17502.86  19492.60  22490.06   5017.8   5512.8   6016.2
